In [81]:
from parser_funcs import *
import numpy as np
import os
import math
from scipy.special import kn
from scipy.interpolate import interp1d
from scipy.interpolate import SmoothBivariateSpline
from scipy.optimize import brentq
from scipy.integrate import quad
from scipy.interpolate import griddata
from scipy.optimize import minimize_scalar
from scipy.optimize import root_scalar
import sys
import glob
import random
import numpy as np
import matplotlib.pyplot as plt

In [67]:
def Event_Parser(event_file, cut):
    with open(event_file) as ef:
        dat=ef.read().splitlines()
        run_num=''
        for line in dat:
            line=line.split()
            if len(line)==2 and line[0]=="Run":
                run_num=line[1]                
                break
    dat2 = [line.split() for line in dat]
    sum_line= dat2[-1]
    weight_tab=[float(line[2]) for line in dat2 if len(line)==3 and line[0]=="event"]
    s_tab=[[float(line[i]) for i in range(1,len(line))] for line in dat2 if len(line)>1\
                and line[0]=='Dark_Scalar']
    lep = [[float(line[i]) for i in range(1,len(line))]+[line[0]] for line in dat2 if len(line)>1\
                and (line[0]=='Decay_Electron' or line[0]=='Decay_Muon')]
    antilep = [[float(line[i]) for i in range(1,len(line))]+[line[0]] for line in dat2 if len(line)>1 and (line[0]=='Decay_Positron' or line[0]=='Decay_Antimuon')]
    ms = sum_line[2]
    eps_q = sum_line[3]
    eps_l = sum_line[4]
    eps_w = sum_line[5]
    nevents = float(sum_line[1])
    POT = float(sum_line[10])
    S_lifetime=hbar/float(sum_line[6])
    eff=float(sum_line[9])
    nevents,weight_tab,s_tab,lep,antilep_tab=cut(nevents,weight_tab,s_tab,lep,antilep)
    return [float(ms),float(eps_q),float(eps_l),float(eps_w),eff,S_lifetime,nevents,weight_tab,s_tab,lep,antilep]

In [3]:
def f(x): ## A function use in the form factor
    if (x<=1):
        return np.arcsin(np.sqrt(x))**2
    else:
        return -1/4*(np.log((1+np.sqrt(1-1/x))/(1-np.sqrt(1-1/x))) - 1j*np.pi)**2

### W boson, quarks and leptons masses in GeV
m_quark_uplike = [2.15e-3, 1.280, 173.21] # u, c, t
m_quark_downlike = [4.7e-3, 0.094, 4.18] # d, s, b
m_lepton = [0.511e-3, 0.1057, 1.777]
m_W, v = 80.38, 246.22
alpha_em = 1/137

def Decay_Width_Photon(m_S,eps_q,eps_l):   # decay width to di-photon through effective di-photon coupling
    eps_W = eps_q   ## an assumption in the model 
    x_w = (m_S/(2*m_W))**2    # mass ratio
    A_W = -(2*x_w**2 + 3*x_w + 3*(2*x_w-1)*f(x_w))/x_w**2     ## A_q, A_l and A_W are loop functions. 
    A_l, A_q = 0, 0
    for m_l in m_lepton:
        x_l = (m_S/(2*m_l))**2
        A_l += 2*(x_l + (x_l-1)*f(x_l))/x_l**2
    for m_q in m_quark_uplike:
        x_q = (m_S/(2*m_q))**2
        A_q += 3*(2/3)**2*(2*(x_q + (x_q-1)*f(x_q))/x_q**2)
    for m_q in m_quark_downlike:
        x_q = (m_S/(2*m_q))**2
        A_q += 3*(1/3)**2*(2*(x_q + (x_q-1)*f(x_q))/x_q**2)   
    Form_Factor_Photon = eps_q*A_q + eps_l*A_l +  eps_W*A_W 
    Gamma_Photon = m_S**3*(alpha_em/(4*np.pi))**2/(16*np.pi*v**2)*np.absolute(Form_Factor_Photon)**2
    return Gamma_Photon

def betha(m_S,m):
    return np.power(1-np.power(2*m/m_S,2),0.5)

def Decay_Width_lepton(m_lepton,m_S,eps_l):  # decay width to pairs of l+l-
    Gamma_lepton = (m_S*eps_l**2*betha(m_S,m_lepton)**3/(8*np.pi))*(m_lepton/v)**2 if (m_S>=2*m_lepton) else  0
    return Gamma_lepton

In [4]:
def Decay_Width_Total(m_S,eps_l,eps_q):
    return Decay_Width_lepton(m_lepton[0],m_S,eps_l)+Decay_Width_lepton(m_lepton[1],m_S,eps_l)+Decay_Width_Photon(m_lepton[0],m_S,eps_l)

In [92]:
test=[[m_S,Decay_Width_Total(m_S,1e-4,1e-2)] for m_S in np.array([190,200,205,210,215,220])*1e-3]

In [98]:
hbar*speed_of_light/Decay_Width_Total(0.2,1e-2,1e-2)

57.6598635418461

In [99]:
hbar*speed_of_light/Decay_Width_Total(0.22,1e-2,1e-2)

0.05766043317926463

In [93]:
test

[[0.19, 3.2566046381982446e-22],
 [0.2, 3.4280509525505073e-22],
 [0.20500000000000002, 3.513775035035174e-22],
 [0.21, 3.5994997508042713e-22],
 [0.215, 9.577241324294905e-20],
 [0.22, 3.427394361316076e-19]]

In [68]:
def Return_True(dat,ds,lep,alep,ms,l):
    return True
#eps=eps_l
def Rescale_Events(dat,ds,lep,alep,ms,lifetime,tot_weight,eps,epset,eps_q,events,cut,state=None):
    if state is not None:
        random.setstate(state)
    prob=0
    #life=lifetime*(epset/eps)**2
    dwtot=Decay_Width_Total(ms,eps,eps_q)
    life=hbar/dwtot
    l=1.0/speed_of_light/life*ms
    for i,u in enumerate(dat):
        ptmp=dec_prob(u[1]*l/u[0],u[2]*l/u[0])
        if cut(u,ds[i],lep[i],alep[i],ms,l):
            prob+=ptmp
        else:
            continue
    return prob/tot_weight*events*(Decay_Width_lepton(m_lepton[0],ms,eps)/dwtot)

In [69]:
def calc_params(file,evnt,cut,cut2=Return_True):
    try:#eps_l is called eps in this
        ms,eps_q,eps_eval,eps_w,eff,S_lifetime,nevents,weight_tab,ds_dat,lep,alep=Event_Parser(file,cut)
    except:
        print("Exception", file)
        return [-1,-1,-1]
    if nevents==0:
        return [-1,-1,-1]
    dat = [[mom(u),u[14],u[15]] for u in ds_dat]
    tot_weight=0
    for i in weight_tab:
        tot_weight+=i
    state=random.getstate()
    evnt_rate = lambda eps : -1.0*Rescale_Events(dat,ds_dat,lep,alep,ms,S_lifetime,tot_weight,10**eps,eps_eval,eps_q,nevents,cut2,state=state)
    diff_from_evnt = lambda eps : abs(evnt_rate(eps)+evnt)
    diff_from_evnt_2 = lambda eps : evnt-Rescale_Events(dat,ds_dat,lep,alep,ms,S_lifetime,tot_weight,eps,eps_eval,eps_q,nevents,cut2,state=state)
    opt=minimize_scalar(evnt_rate,bounds=(-8,0),method='bounded')
    if opt.fun>-evnt:
        return [ms,-1,-1]
    
    opt2=minimize_scalar(diff_from_evnt,bounds=(-8,opt.x),method='bounded')

    x=opt.x
    xstep=0.1
    while diff_from_evnt_2(10**x)<0:
        x+=xstep
    
    sol = root_scalar(diff_from_evnt_2, bracket=[10**(x-xstep), 10**x], method='brentq')    
    
    return [ms,10**opt2.x,sol.root]

In [28]:
Nucal_efficiency=0.7
Nucal_energy_cut_1=3
Nucal_evnt_1=6.06
Nucal_angle=0.05
def Nucal_Cut_1(nevents,weight_tab,S_tab,lep_tab,antilep_tab):
    pass_index=[]
    for i in range(len(weight_tab)):
        energy=lep_tab[i][0]+antilep_tab[i][0]
        if lep_tab[i][-1]=="Decay_Electron" and energy>Nucal_energy_cut_1 and theta(lep_tab[i]) < Nucal_angle and theta(antilep_tab[i]) < Nucal_angle:
            pass_index.append(i)
    nevents*=len(pass_index)/len(weight_tab)*Nucal_efficiency
    weight_tab = [weight_tab[i] for i in pass_index]
    S_tab = [S_tab[i] for i in pass_index]
    lep_tab = [lep_tab[i] for i in pass_index]
    antilep_tab = [antilep_tab[i] for i in pass_index]
    return nevents,weight_tab,S_tab,lep_tab,antilep_tab

Nucal_energy_cut_2=10
Nucal_evnt_2=3.64
def Nucal_Cut_2(nevents,weight_tab,S_tab,lep_tab,antilep_tab):
    pass_index=[]
    for i in range(len(weight_tab)):
        energy=lep_tab[i][0]+antilep_tab[i][0]
        if lep_tab[i][-1]=="Decay_Electron" and energy>Nucal_energy_cut_2 and theta(lep_tab[i]) < Nucal_angle and theta(antilep_tab[i]) < Nucal_angle:
            pass_index.append(i)
    nevents*=len(pass_index)/len(weight_tab)*Nucal_efficiency
    weight_tab = [weight_tab[i] for i in pass_index]
    S_tab = [S_tab[i] for i in pass_index]
    lep_tab = [lep_tab[i] for i in pass_index]
    antilep_tab = [antilep_tab[i] for i in pass_index]
    return nevents,weight_tab,S_tab,lep_tab,antilep_tab

In [95]:
nucal_files=glob.glob('Dark_Scalar/nucal/*')
bounds=[calc_params(file,Nucal_evnt_1,Nucal_Cut_1) for file in nucal_files]
bounds = [x for x in bounds if x[0]!=-1 and (x[1]!=-1 and x[2]!=-1)]
bounds.sort()
record_list("Scalar_Nucal.dat",bounds)

In [ ]:
calc_params('Dark_Scalar/nucal/nucal_0.1.dat',Nucal_evnt_1,Nucal_Cut_1)

In [80]:
bounds

[[0.0011, 0.1241540559862014, 458.0858830569868],
 [0.0012, 0.06628960209154672, 252.7243181801338],
 [0.0013, 0.048376508175751655, 181.71318186414408],
 [0.0014, 0.03796511141793081, 144.93799061075978],
 [0.0015, 0.032771544521343336, 120.60571226028864],
 [0.0017, 0.025000885596184273, 93.39954480174201],
 [0.0019, 0.020445345446396743, 77.46012066460115],
 [0.002, 0.018634328880530363, 71.48995696066628],
 [0.003, 0.010923499113932691, 41.75562243255692],
 [0.004, 0.007875674254798744, 30.008612364254436],
 [0.005, 0.006131046881623625, 23.530579235520754],
 [0.01, 0.0030060150449903026, 11.504173096662232],
 [0.015, 0.002000456381338038, 7.637857425409387],
 [0.02, 0.0015067478079864973, 5.7088239786268],
 [0.03, 0.0010111080063151339, 3.8057457942214863],
 [0.04, 0.0007620108027406087, 2.857939603143908],
 [0.06, 0.0005212718029298849, 1.9021306637242335],
 [0.08, 0.00040521343313971677, 1.4268403285456297],
 [0.1, 0.00033659530337275804, 1.1419281703405746],
 [0.13, 0.000276859